### The same code fromrecommendation_system.py

In [35]:
import pickle
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
# CountVectorizer
with open('finalized_model.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# Already vectorized all the movies in dataset of recommendations
vectorized_dataset = sparse.load_npz('vectorized_dataset.npz')

# Dataset with all data about movie for recommendations
df_full = pd.read_csv("finalized_dataset.csv")

In [37]:
def recommend_movie(liked_movies):
    # get prepared dataset of liked movies
    df_liked_movies = df_full[df_full['tconst'].isin(liked_movies["imdb_id"])]
    user_movie_idx = df_liked_movies.index

    # retrieve soup from actors/directors/writes ids + genres
    movies_soup = df_liked_movies['soup']
    # transform soup to sparse matrix of vectorized values
    count_user_matrix = vectorizer.transform(movies_soup)
    # sum up vectors and normalize it
    count_user_vec = count_user_matrix.sum(axis=0) / count_user_matrix.sum(axis=0).max()

    # calculate similarity
    similarity = cosine_similarity(sparse.csr_matrix(count_user_vec), vectorized_dataset)[0]
    # add info about average rating from all users, to let top movies be higher
    scores = similarity * df_full['weightedAverageScales']

    # sort similarities
    similar_scores = list(enumerate(scores))
    similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)

    # retrieve top 10 idx movies to recommend
    recommend_movie_indices = [idx for idx, score in similar_scores if idx not in user_movie_idx][:10]
    return df_full.loc[recommend_movie_indices]

In [38]:
# Reset index of our main DataFrame and construct reverse mapping as before
indices = pd.Series(df_full.index, index=df_full['originalTitle']).to_dict()
tconsts = df_full[['tconst', "originalTitle"]].set_index("originalTitle").to_dict()["tconst"]

In [43]:
values = [
    # (tconsts["Avatar"]),
    
    # (tconsts["La La Land"]),
    # (tconsts["The Help"]),
    # (tconsts["Easy A"]),
    # 
    (tconsts["Inception"]),
    (tconsts["Tenet"]),
    (tconsts["Interstellar"]),
]
liked_movies = pd.DataFrame(values, columns=["imdb_id"])
liked_movies

,imdb_id
0,tt1375666
1,tt6723592
2,tt0816692


In [44]:
recommend_movie(liked_movies)

,tconst,originalTitle,weightedAverageScales,soup
10700,tt1392190,Mad Max: Fury Road,0.870526,action adventure sci-fi nm0004306 nm0362766 nm...
14437,tt4154756,Avengers: Infinity War,0.902761,action adventure sci-fi nm0751577 nm0751648 nm...
3845,tt0103064,Terminator 2: Judgment Day,0.924163,action adventure sci-fi nm0000116 nm0000216 nm...
8233,tt0482571,The Prestige,0.913645,drama mystery sci-fi nm0634240 nm0000288 nm041...
10566,tt1345836,The Dark Knight Rises,0.903098,action drama thriller nm0634240 nm0000288 nm03...
2916,tt0090605,Aliens,0.902194,action adventure sci-fi nm0000116 nm0000244 nm...
6679,tt0316654,Spider-Man 2,0.805923,action adventure sci-fi nm0000600 nm0001497 nm...
12178,tt1877832,X-Men: Days of Future Past,0.848720,action adventure sci-fi nm0001741 nm0001772 nm...
5442,tt0145487,Spider-Man,0.795417,action adventure sci-fi nm0000600 nm0001497 nm...
12861,tt2250912,Spider-Man: Homecoming,0.795253,action adventure sci-fi nm1218281 nm4043618 nm...


In [8]:
df_liked_movies = df_full[df_full['tconst'].isin(liked_movies["imdb_id"])]
user_movie_idx = df_liked_movies.index

# retrieve soup from actors/directors/writes ids + genres
movies_soup = df_liked_movies['soup']
# transform soup to sparse matrix of vectorized values
count_user_matrix = vectorizer.transform(movies_soup)
# sum up vectors and normalize it
count_user_vec = count_user_matrix.sum(axis=0) / count_user_matrix.sum(axis=0).max()

# calculate similarity
similarity = cosine_similarity(sparse.csr_matrix(count_user_vec), vectorized_dataset)[0]
# add info about average rating from all users, to let top movies be higher
scores = similarity * df_full['weightedAverageScales']

# sort similarities
similar_scores = list(enumerate(scores))
similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)

# retrieve top 10 idx movies to recommend
recommend_movie_indices = [idx for idx, score in similar_scores if idx not in user_movie_idx][:10]
# return df_full.loc[recommend_movie_indices]

/tmp/ipykernel_16261/3413097452.py:9: RuntimeWarning: invalid value encountered in divide
  count_user_vec = count_user_matrix.sum(axis=0) / count_user_matrix.sum(axis=0).max()


ValueError: Input contains NaN.

In [34]:
df_full

,tconst,originalTitle,weightedAverage,soup
0,tt0004972,The Birth of a Nation,6.112228,drama history war nm0000428 nm0001273 nm055061...
1,tt0006206,Les vampires,6.761700,action adventure crime nm0275421 nm0615736 nm0...
2,tt0006864,Intolerance: Love's Struggle Throughout the Ages,7.349504,drama history nm0000428 nm0001273 nm0366008 nm...
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,6.881505,drama romance nm0000428 nm0001273 nm0001932 nm...
4,tt0010323,Das Cabinet des Dr. Caligari,7.878620,horror mystery thriller nm0927468 nm0470328 nm...
...,...,...,...,...
16751,tt9894470,VFW,6.126264,action crime horror nm3976308 nm0002332 nm0006...
16752,tt9898858,Coffee & Kareem,5.449330,action comedy crime nm0236226 nm1159180 nm0378...
16753,tt9900782,Kaithi,8.165990,action crime drama nm7992231 nm1912683 nm12308...
16754,tt9907782,Eight for Silver,6.194643,fantasy horror mystery nm1193346 nm2933542 nm0...
